In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

In [2]:
f = open("uw_event_data.csv", "w")
headers = "event_name,date,time,day_of_week\n"
f.write(headers)

url = "https://www.washington.edu/calendar/"
iframe_id = "trumba.spud.1.iframe"

driver = webdriver.Chrome()

try:
    driver.get(url)
    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, iframe_id))
    )
    driver.switch_to.frame(iframe)

    num_pages_to_scrape = 10

    for _ in range(num_pages_to_scrape):
        try:
            # Add Beautiful Soup code here
            content = driver.page_source
            soup = BeautifulSoup(content, 'html.parser')

            event_links = soup.find_all('a', attrs={'onmouseout': re.compile(r'Trumba.Spuds.popupOnMouseOut.*'),
                                                     'onmouseover': re.compile(r'Trumba.Spuds.popupOnMouseOver.*'),
                                                     'url.linktype': 'EventDetail'})

            event_names = [link.get_text(strip=True) for link in event_links]

            for event_name in event_names:
                f.write(event_name + "," + "," + "," + "\n")
                
            # Wait for the button to be clickable
            button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "ctl04_ctl07_ctl00_lnk2PrevPg"))
            )
            button.click()

            # Wait for the content to change
            WebDriverWait(driver, 10).until(
                EC.staleness_of(iframe)
            )

        except StaleElementReferenceException:
            print("StaleElementReferenceException: Retry clicking the button.")
            continue

finally:
    driver.quit()
    f.close()

StaleElementReferenceException: Retry clicking the button.


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x000000010523e004 chromedriver + 4169732
1   chromedriver                        0x0000000105235ff8 chromedriver + 4136952
2   chromedriver                        0x0000000104e8b500 chromedriver + 292096
3   chromedriver                        0x0000000104ed07a0 chromedriver + 575392
4   chromedriver                        0x0000000104f0b818 chromedriver + 817176
5   chromedriver                        0x0000000104ec45e8 chromedriver + 525800
6   chromedriver                        0x0000000104ec54b8 chromedriver + 529592
7   chromedriver                        0x0000000105204334 chromedriver + 3932980
8   chromedriver                        0x0000000105208970 chromedriver + 3950960
9   chromedriver                        0x00000001051ec774 chromedriver + 3835764
10  chromedriver                        0x0000000105209478 chromedriver + 3953784
11  chromedriver                        0x00000001051deab4 chro

In [3]:
f = open("uw_football_data_all.csv", "w")
headers = "date,year_dayofweek,time,at,opponent,location_city,location_state,tv,radio,result_1,result_2,links\n"
f.write(headers)

base_url = "https://gohuskies.com/sports/football/schedule/"

for year in range(2019, 2024):
    url = f"{base_url}{year}?grid=true"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        tr_elements = soup.find_all('td')
        td_elements = []

        for td in tr_elements:
            text = td.get_text(strip=True)
            td_elements.append(text)
    
        if year == 2019 or year == 2022:
        # 2019 and 2022 has an extra column 'Tournament', skip that column
            for i in range(0, len(td_elements), 10):
                chunk = td_elements[i:i + 10]
                f.write(','.join(chunk[:8] + chunk[9:]) + '\n')
        else:
            for i in range(0, len(td_elements), 9):
                chunk = td_elements[i:i + 9]
                f.write(','.join(chunk) + '\n')  
            
f.close()

In [4]:
uw_football_data = pd.read_csv("uw_football_data_all.csv")
uw_football_data = uw_football_data.iloc[:, : 7]
uw_football_data

,date,year_dayofweek,time,at,opponent,location_city,location_state
0,August 31,2019 (Saturday),12:00 PM PT,Home,Eastern Washington,Seattle,WA
1,September 7,2019 (Saturday),7:30 PM PT,Home,California,Seattle,WA
2,September 14,2019 (Saturday),4:30 PM PT,Home,Hawai'i,Seattle,WA
3,September 21,2019 (Saturday),12:30 PM PT,Away,Brigham Young,Provo,UT
4,September 28,2019 (Saturday),12:30 PM PT,Home,USC,Seattle,WA
5,October 5,2019 (Saturday),7:30 PM PT,Away,Stanford,Stanford,CA
6,October 12,2019 (Saturday),8:00 PM PT,Away,Arizona,Tucson,AZ
7,October 19,2019 (Saturday),12:30 PM PT,Home,Oregon,Seattle,WA
8,November 2,2019 (Saturday),1:00 PM PT,Home,Utah,Seattle,WA
9,November 8,2019 (Friday),7:30 PM PT,Away,Oregon State,Corvallis,OR
